In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

cwd = os.getcwd()  # get directory for storage

# This file automates the entire pipeline for assertion generation with chatgpt

## Step 1) Get Asserted Code From Github

### Step 1.1) Clean and process the code
### Step 1.2) Extract Ground-Truth Assertions & Relevant Statistics

In [2]:
from google.cloud import bigquery as bq

def get_asserted_code(num=100000, ext="%.py", verbose=True):
    query_string = """SELECT f.repo_name, c.content
FROM `bigquery-public-data.github_repos.files` AS f
JOIN `bigquery-public-data.github_repos.contents` AS c
ON f.id = c.id
WHERE
NOT c.binary
AND f.path LIKE '%.py'
AND REGEXP_CONTAINS(c.content, r'(?m)^\s*assert ')
LIMIT """ + str(num)
    
    if isinstance(num, int):
        secret_dir = "Data/secret/"
        api_key = cwd + "/" + secret_dir + os.listdir(secret_dir)[0]
        assert api_key[-5:] == ".json"  # confirm that it was found
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_key
        query_string = query_string.replace("%.py", ext)

        if verbose:
            print("*Running Query:")
            print(query_string)
            print()
        client = bq.Client()
        df = (
            client.query(query_string)
            .result()
            .to_dataframe(
                create_bqstorage_client=True,
            )
        )
    elif isinstance(num, str):
        # load data from file
        df = pd.read_csv(num)
        print("Found data at", num)
    else:
        print("first param type undefined, must be string signifying directory of csv or\
               int signifying number of records to scrib from bigquery...")
        assert False
    
    if verbose:
        print("*Handling Duplicates...")
    init_len = len(df)
    df.drop_duplicates(subset=["content"], keep="first", inplace=True)
    if verbose:
        print("#Non-duplicates / #Total Retrieved =", (len(df)/init_len))
    return df

verilog_dir = cwd+"/Data/BigQuery/VerilogAssertions-ALL.csv"
python_dir = cwd+"/Data/BigQuery/PythonAssertions100k.csv"
# df = get_asserted_code(python_dir)  # 10
# df

In [32]:
conditionals = dict([[cond, i] for i, cond in enumerate(["==", "!=", "<=", ">=", "<", ">"])])
compounding_statements = ["and"]
bad_statements = [" or ", " in ", "isinstance"]  # TODO: properly account for OR
def parse_assertions(func, is_split=True, verbose=False):
    """
    Format: "assert [expression], [return_string]"
    
    Exceptions to Handle:
    - 'in'/'not in' keyword
    - boolean functions - ex. isinstance(var, type)
    - separation of attributes - ex. len(var), var[i]
    """
#     if verbose:
#         print("*Extracting Assertions...")
    out = []
    asserted_lines = 0
    lines = []
    for temp in func.split('\n'):  # find lines with assert in them
        if "assert" in temp:
            asserted_lines += 1
            bad_flag = False
            for bad in bad_statements:
                if bad in temp:
                    bad_flag = True
            if not bad_flag:
                lines.append(temp.strip())
    # TODO: experiment with smaller content window for assertions
    ind = 0
    while ind < len(lines):
        data = lines[ind].strip()
        start = data.find('assert')
        if start == -1:  # double checking that the assertion exists in this line
            ind += 1
            continue
        # account for combination statements
        for statement in compounding_statements:
            add_statement = data.find(statement)
            if add_statement != -1:
                extra_line = data[add_statement+len(statement):]
                lines.insert(ind+1, "assert "+extra_line)
                data = data[:add_statement].strip()

        com = data.find(',')   # parsing out return_string
        if com != -1:
            data = data[:com]
        com = data.find('#')
        if com != -1:   # parsing out comments
            if com < start:  # if the assertion itself is a comment
                ind += 1
                continue
            else:
                data = data[:com]

        if is_split:  # splitting the assertion into components for analysis
            data = [var.strip() for var in data.split(' ') if len(var.strip()) > 0]
            
            if len(data) < 1:  # edge case: nothing after 'assert' (likely typo)
                if verbose:
                    print("empty assertion found?: ", data, '\n', lines[ind])
                ind += 1
                continue
                
            if data[0] != "assert":  # edge case: something before the 'assert' statement
                ind += 1
#                 if verbose:
#                     print("something was found before the assertion on this line:\n", data)
                continue
    
            data = data[1:]  # from here on we only care about the content after the 'assert' keyword
            if len(data) < 1:  # edge case: nothing after 'assert' (likely typo)
                if verbose:
                    print("empty assertion found?: ", data, '\n', lines[ind])
                ind += 1
                continue

            condition = True  # assertion [variable] == condition by default
            if data[0] == "not":  # accounting for 'not' keyword
                condition = False
                data = data[1:]
            
            if len(data) == 1:  # adding == to simlify
                data = data + ["==", str(condition)]

            for i in range(len(data)):
                if data[i] == "is":  # simplifying is to ==
                    data[i] = "=="
                if data[i] in conditionals.keys():  # parsing common conditionals
                    data = [' '.join(data[:i]), data[i], ' '.join(data[i+1:])]  # conditionals[data[i]]
                    break

        if verbose and len(data) != 3:
            print("Weird assertion found:\n", data, '\n', lines[ind])
            print()
#             assert len(data) == 3, "found conditional-less assertion:\n" + str(data) + '\n' + str(lines[ind-1:ind+2])
        else:
            out.append(data)
        ind += 1
    return out, asserted_lines

def unassert(code, delim='', add_count=True):
    out = ""
    counter = 1
    for line in code.split('\n'):
        if "assert" not in line:
            out += '\n'
            if add_count:
                out += str(counter)
            out += delim+line
            counter += 1
    return out

def get_assertion(temp_df, verbose=False, unassert_col=True, add_stats=True):
    """ run assertion generation """
    # tester_df["assertions"] = tester_df["content"].apply(lambda code: get_assertions(code))
    
    assertions = []  # list of parsed assertions
    asserted_lines = []  # number of lines with 'assert' in them
    parsed_lines = []  # number of assertions easily parsed
    arr = []  # assertion recovery ratio
    atl = []  # assertions to size
    for i, row in tqdm(temp_df.iterrows()):
        parsed, lines = parse_assertions(row["content"], True, verbose)
        assertions.append(parsed)
        asserted_lines.append(lines)
        parsed_lines.append(len(parsed))
        arr.append(len(parsed)/lines)
        atl.append(len(parsed)/len(row["content"]))

    if unassert_col:
        temp_df["unasserted"] = temp_df["content"].apply(lambda code: unassert(code))
    
    if add_stats:
        temp_df["assertions"] = assertions
        temp_df["asserted_lines"] = asserted_lines
        temp_df["parsed_lines"] = parsed_lines
        temp_df["arr"] = arr
        temp_df["atl"] = atl
    return temp_df

# tester_df = df.copy()
# tester_df = get_assertion(tester_df)
# tester_df

## Step 2) Generate LLM Prompt & Query a GPT

In [9]:
banned_vars = ['', '*', 'self']
def old_get_variables(func, verbose=False):
    out = []
    for line in func.split('\n'):
        line = line.strip()
        if "def " in line:  # add params if its a function
            start = line.find('(')
            end = line.find(')')
            for new_param in line[start+1:end].split(','):
                default = new_param.find("=")
                if default != -1:
                    new_param = new_param[:default]
                new_param = new_param.strip()
                if new_param not in out and new_param not in banned_vars:
                    if verbose:
                        print("*Found  {", new_param, "}  at:\n", line, '\n')
                    out.append(new_param)
        else: # add variables if equals operation
            find_var = line.find(' = ')
            if find_var != -1:
                new_var = line[:find_var].strip()
                
                if ',' in new_var: # handle tuple equalities edge case (ex: a, b, c = fn_output())
                    var_list = [tuple_var.strip() for tuple_var in new_var.split(',')]
                else:
                    var_list = [new_var]
                for new_var in var_list:
                    if new_var not in out and new_var not in banned_vars:
                        if verbose:
                            print("**Found  {", new_var, "}  at:\n", line, '\n')
                        out.append(new_var)
            # TODO: handle indexing
    return out


# test
import ast

def get_variables(code):  # TODO: run a proper test
    tree = ast.parse(code)
    variables = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    variables.append(target.id)
    return variables

def get_all_variables(df):  # TODO test
    ret = []
    for i, row in tqdm(df.iterrows()):
        try:
            these_vars = get_variables(row["content"])
        except:
            these_vars = old_get_variables(row["content"])
        for statement in row["assertions"]:
            if len(statement) > 0:
                new_var = str(statement[0])
                if new_var not in these_vars:
                    these_vars.append(new_var)
        ret.append(these_vars)
    return ret

# out = get_variables(df.sample()["content"].iloc[0])
# get_vars = lambda code: get_variables(code)
# tester_df["variables"] = tester_df["content"].apply(get_vars)
# tester_df

In [5]:
# from ipynb.fs.full.Data.GitHub-Assertions import get_variables
class prompt_example:
        def __init__(self, this_in="", this_out=""):
            self.input = this_in
            self.output = this_out
            
        def composite(self):
            return "Example Input:\n" + self.input + "\nExample Output:\n" + self.output
        
class LLM_prompt:       
    def __init__(self, input_code="*Variables:\n[flag, num, i]\n*Code:\n1num = int(input(\"Enter a number: \"))  # Program to check if a number is prime or not\n2flag = False  # define a flag variable\n3\n4if num == 1:\n5    print(num, \"is not a prime number\")\n6elif num > 1: # check for factors\n7    for i in range(2, num):\n8        if (num % i) == 0:\n9            flag = True  # if factor is found, set flag to True\n10            break  # break out of loop\n11    if flag:  # check if flag is True\n12        print(num, \"is not a prime number\")\n13    else:\n14        print(num, \"is a prime number\")",
                 example_in="*Variables:\n[n]\n*Code:\n1def fibonacci(n):\n2   if n <= 1:\n3       return n\n4   else:\n5       return(recur_fibo(n-1) + recur_fibo(n-2))",
                 example_out="[1, n, >=, 1, \"the fibonacci sequence can only be done on posative integers\"]\n\nWhich would be the same as:\n1def fibonacci(n):\n2   assert n >= 1\n3   if n <= 1:\n4       return n\n5   else:\n6       return(recur_fibo(n-1) + recur_fibo(n-2))", 
                 criteria=["Assert that the function can take in all inputs necessary to complete the process",
                           "Assert that all outputs are of the proper sizes."]
                 ):
        self.criteria = criteria
        self.example = prompt_example(example_in, example_out)
        self.input_code = input_code
        
        # default params that are less likely to change
        self.intro = "You are a helpful bot that adds assertions to pieces of Python code."  
        self.input_format = "You will be given a list of variables and a string of code presented in the format:\n*Variables:\n[...]\n*Code:\n..."
        self.criteria_transition = "Generate assertions based on the following criteria:"
        self.output_format = "Your response should ONLY be a list of assertions in the format:\n[line_number, subject_variable, condition_type, target, reasoning]"
        self.output_format_description = ["line_number is an integer referencing the line after which the assertion should be inserted",
                                          "subject_variable and target can ONLY be variables from the input list, integers, booleans, or None", # TODO retest bools
                                          "condition_type can only be a value in this list: [==, >=, <=, !=]",
                                          "reasoning is a short decription of why the assertion was made"]
        self.example_transition = "Here is an example of what your input will look like and what you should return:"
        self.input_transition = "Here is the actual input you should provide assertions for:"
    
    
    
    def composite_criteria(self):
        """ return criteria as a single string"""
        ret = ""
        for i, crit in enumerate(self.criteria):
            ret += str(i+1) + ") " + crit
            if i != len(self.criteria)-1:  # ignore last instance for formatting
                ret += '\n'
        return ret
    
    def composite_output_formatting(self):
        ret = self.output_format
        for desc in self.output_format_description:
            ret += "\n -" + desc 
        return ret
    
    def prompt(self):
        """ return entire prompt"""
        return '\n'.join([self.intro, self.input_format,
                          self.criteria_transition, self.composite_criteria(), "",
                          self.composite_output_formatting(), "",
                          self.example_transition, self.example.composite(), "\n",
                          self.input_transition, self.input_code])
    
    def to_list(self):
        """ return key prompt components as a list """
        return [self.intro, self.formatting, self.criteria, self.example, self.input_code, self.prompt()]
    
    def __str__(self):
        return self.prompt()
    def __repr__(self):
        return self.prompt()

tester = LLM_prompt()
print(len(str(tester)))
tester

# fib_input = "def fibonacci(n):\nassert n >= 1\nif n <= 1:\nreturn n\nelse:\nreturn(recur_fibo(n-1) + recur_fibo(n-2))"
# fib_output = "[1, n, 1, 1, the fibonacci sequence can only be done on posative integers]"

1931


You are a helpful bot that adds assertions to pieces of Python code.
You will be given a list of variables and a string of code presented in the format:
*Variables:
[...]
*Code:
...
Generate assertions based on the following criteria:
1) Assert that the function can take in all inputs necessary to complete the process
2) Assert that all outputs are of the proper sizes.

Your response should ONLY be a list of assertions in the format:
[line_number, subject_variable, condition_type, target, reasoning]
 -line_number is an integer referencing the line after which the assertion should be inserted
 -subject_variable and target can ONLY be variables from the input list, integers, booleans, or None
 -condition_type can only be a value in this list: [==, >=, <=, !=]
 -reasoning is a short decription of why the assertion was made

Here is an example of what your input will look like and what you should return:
Example Input:
*Variables:
[n]
*Code:
1def fibonacci(n):
2   if n <= 1:
3       return

In [6]:
def make_prompts(temp_df):
    prompts = []
    for i, row in tqdm(temp_df.iterrows()):
        # *Variables:\n[flag, num, i]\n*Code:\n
        prompt_param = "*Variables:\n" + str(row["variables"]) + "\n*Code:\n" + row["unasserted"]
        prompts.append(str(LLM_prompt(prompt_param)))
    temp_df["prompt"] = prompts
    temp_df["prompt_len"] = [len(p) for p in prompts]
    return temp_df
# tester_df = make_prompts(tester_df)
# tester_df

In [49]:
# querying
import openai
import altair as alt
import json
from vega_datasets import data

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
openai.api_key = OPENAI_API_KEY
def run_gpt4(messages):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages
    )
#     return response["choices"][0]["message"]["content"]
    return response



def gpt_oneshot(input_prompt, directive="You are a helpful bot that adds assertions to pieces of Python code.", verbose=False):
    message_hist = [{"role": "system", "content": directive},  # add directed
                    {"role": "user", "content": input_prompt}]  # init
    response = run_gpt4(message_hist)
#     if verbose:
#         print("chat_gpt: ", response, '\n')
#     message_hist.append({"role": "system", "content": response})
    return response

tester = gpt_oneshot("Write python code to sort n numbers")

In [170]:
df = get_asserted_code(cwd+"/Data/BigQuery/PythonAssertions100k.csv", "%.py", False)
df = get_assertion(df)
df

Found data at /Users/korahughes/Documents/GitHub/LLMCodeGen/Data/BigQuery/PythonAssertions100k.csv


33793it [00:06, 4947.82it/s]


,repo_name,content,unasserted,assertions,asserted_lines,parsed_lines,arr,atl
0,tqchen/tvm,# Licensed to the Apache Software Foundation (...,\n1# Licensed to the Apache Software Foundatio...,"[[len(glbs), ==, 3], [len(result), ==, 2], [le...",29,4,0.137931,0.000376
1,Lujeni/ansible,# (c) 2017 Red Hat Inc.\n#\n# This file is par...,\n1# (c) 2017 Red Hat Inc.\n2#\n3# This file i...,"[[exc_info.match('FAIL'), ==, True], [result['...",29,20,0.689655,0.002668
2,lukas-hetzenecker/home-assistant,"""""""The tests for the Pilight sensor platform.""...","\n1""""""The tests for the Pilight sensor platfor...","[[await, async_setup_component(], [state.state...",14,9,0.642857,0.002150
3,schnoebe/fedora-mock,import fcntl\nimport glob\nimport grp\nimport ...,\n1import fcntl\n2import glob\n3import grp\n4i...,"[[our_dir, ==, True]]",1,1,1.000000,0.000044
4,samstav/fastfood,# -*- coding: utf-8 -*-\n# Copyright 2015 Rack...,\n1# -*- coding: utf-8 -*-\n2# Copyright 2015 ...,"[[len(key_val), ==, 2]]",1,1,1.000000,0.000115
...,...,...,...,...,...,...,...,...
33788,raphaelm/django-i18nfield,from i18nfield.admin import I18nModelAdmin\nfr...,\n1from i18nfield.admin import I18nModelAdmin\...,"[[admin.formfield_overrides[I18nCharField], ==...",4,1,0.250000,0.001441
33789,fniephaus/alfred-rworkflow,# The MIT License (MIT)\n#\n# Copyright (c) 20...,\n1# The MIT License (MIT)\n2#\n3# Copyright (...,"[[offset, >, 0]]",1,1,1.000000,0.000223
33790,bgris/ODL_bgris,# -*- coding: utf-8 -*-\r\n#\r\n# Copyright © ...,\n1# -*- coding: utf-8 -*-\r\n2#\r\n3# Copyrig...,[],1,0,0.000000,0.000000
33791,chrsrds/scikit-learn,"""""""\nTesting for the base module (sklearn.ense...","\n1""""""\n2Testing for the base module (sklearn....","[[3, ==, len(ensemble)], [3, ==, len(ensemble....",20,18,0.900000,0.003563


In [ ]:
temp = df.iloc[0]["assertions"]
print(type(temp[0]))
print(temp[0][0])

# RUNNING 1SHOT CODE

In [8]:
def one_shot_prompts(my_dir=cwd+"/Data/BigQuery/PythonAssertions100k.csv", ext="%.py"):
    print("GETTING CODE")
    df = get_asserted_code(my_dir, ext, False)
    
    print("\nEXTRACTING ASSERTIONS")
    df = get_assertion(df)
    
    all_prompts = len(df)
    df = df[df["parsed_lines"]!=0]
    all_prompts = 100*len(df)/all_prompts
    print("dropping code with no parsed assertions =>", str(all_prompts)+'%')
    
    print("\nEXTRACTING VARIABLES")
#     get_vars = lambda code: get_variables(code)
#     df["variables"] = df["content"].apply(get_vars)
    df["variables"] = get_all_variables(df)
    
    df["num_vars"] = df["variables"].apply(lambda var: len(var))
    all_prompts = len(df)
    df = df[df["num_vars"] > 0]
    all_prompts = 100*len(df)/all_prompts
    print("dropping code with no extracted variables =>", str(all_prompts)+'%')
    
    print("\nGENERATING PROMPTS")
    df = make_prompts(df)
    
    prompt_limit = 8192
    all_prompts = len(df)
    df = df[df["prompt_len"] < prompt_limit]
    all_prompts = 100*len(df)/all_prompts
    print("dropping prompts over limit =>", str(all_prompts)+'%')
    
    df.to_csv(cwd+"/Data/python_prompts_noresponse3.csv") # saving data
    print("Data checkpoint saved...\n")
    
#     print("GENERATING RESPONSES")
#     responses = []
#     for prompt in tqdm(df["prompt"]):
#         responses.append(gpt_oneshot(prompt))
#     df["gpt"] = responses
#     df.to_csv(cwd+"/python_prompts_withresponse.csv") # saving data
    
    return df

df = one_shot_prompts()

GETTING CODE
Found data at /Users/korahughes/Documents/GitHub/LLMCodeGen/Data/BigQuery/PythonAssertions100k.csv

EXTRACTING ASSERTIONS


33793it [00:06, 5053.96it/s]


dropping code with no parsed assertions => 89.8351729648152%

EXTRACTING VARIABLES
dropping code with no extracted variables => 100.0%

GENERATING PROMPTS


30358it [00:02, 13359.44it/s]


dropping prompts over limit => 48.35298768034785%
Data checkpoint saved...



## PARTIAL EXECUTION OF 1SHOT

In [10]:
part_size = 200  # should take about a bit under an hour
current_size = 14464
def partial_execution(part=1):  # slices of 10xt
    start = (part-1)*part_size
    end = start+part_size
#     if start > current_size:
#         return
#     elif end > current_size:
#         end = current_size+1
    df = pd.read_csv(cwd+"/Data/python_prompts_noresponse.csv").iloc[start:end, 1:]
    print("\nGenerating prompts for indexes", start, "to", end)
    
    responses = []
    for prompt in tqdm(df["prompt"]):
        responses.append(gpt_oneshot(prompt))
    df["gpt"] = responses
    
    df.to_csv((cwd+"/Data/Testing/python_prompts_withresponse_part"+str(part)+".csv"), index=False) # saving data
    print("Saved to:", (cwd+"/Data/Testing/python_prompts_withresponse_part"+str(part)+".csv"))
    return df

df = partial_execution(2)
# for i in range(1, 10):  # up to 73
#     print(partial_execution(i))
#     print('\n')


Generating prompts for indexes 200 to 400


100%|████████████████████████████████████████████| 200/200 [1:12:19<00:00, 21.70s/it]


Saved to: /Users/korahughes/Documents/GitHub/LLMCodeGen/Data/Testing/python_prompts_withresponse_part2.csv


In [ ]:
""" Execution Notes:
- somewhere around 320 instances I get various API errors:
'APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 13 Feb 2024 09:21:48 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '854c009eade719c3-EWR', 'alt-svc': 'h3=":443"; ma=86400'}'

"""

In [11]:
df = pd.read_csv(cwd+"/Data/Testing/python_prompts_withresponse_part2.csv").sort_values("prompt_len", ascending=True)
df

,repo_name,content,unasserted,assertions,asserted_lines,parsed_lines,arr,atl,variables,num_vars,prompt,prompt_len,gpt
135,weidenba/recovery_sort,from filter_system.rename import RenameFilterS...,\n1from filter_system.rename import RenameFilt...,"[['len(filter_system.filter_plugins.keys())', ...",2,2,1.000000,0.006780,['filter_system'],1,You are a helpful bot that adds assertions to ...,1582,"[[5, 'filter_system', '!=', None, ""Filter syst..."
28,algorythmic/bash-completion,import pytest\n\n\nclass TestPydoc:\n @pyte...,\n1import pytest\n2\n3\n4class TestPydoc:\n5 ...,"[['completion', '==', 'True'], ['completion', ...",2,2,1.000000,0.007576,['completion'],1,You are a helpful bot that adds assertions to ...,1618,"[[6, 'completion', '!=', None, 'input variable..."
72,SBRG/ome,# -*- coding: utf-8 -*-\n\nfrom cobradb.models...,\n1# -*- coding: utf-8 -*-\n2\n3from cobradb.m...,"[['load_the_map(None', '==', 'True']]",1,1,1.000000,0.003086,"['test_db', 'session']",2,You are a helpful bot that adds assertions to ...,1684,"[[8, 'test_db', '!=', None, ""the test_db shoul..."
150,totalgood/twote,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,\n1#!/usr/bin/env python\n2# -*- coding: utf-8...,"[['fib(1)', '==', '1'], ['fib(2)', '==', '1'],...",3,3,1.000000,0.009434,"['__author__', '__copyright__', '__license__']",3,You are a helpful bot that adds assertions to ...,1696,"[14, '-10', >=, 0, ""the fibonacci sequence can..."
97,AartGoossens/athletic_pandas,import pandas as pd\nimport pytest\n\nfrom ath...,\n1import pandas as pd\n2import pytest\n3\n4fr...,"[[""model.params['hr_rest'].value"", '==', '0.00...",7,7,1.000000,0.009986,"['heartrate', 'power', 'model', 'predictions']",4,You are a helpful bot that adds assertions to ...,1720,"[[8, 'heartrate', '==', 50, ""heartrate should ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,tbenthompson/cppimport,import contextlib\nimport copy\nimport logging...,\n1import contextlib\n2import copy\n3import lo...,"[['p.returncode', '==', 'returncode'], ['os.pa...",15,12,0.800000,0.001981,"['root_logger', 'handler', 'formatter', 'filen...",27,You are a helpful bot that adds assertions to ...,7725,"[[39, 'test_code', '!=', '', 'Check that test_..."
29,mattwthompson/mdtraj,##############################################...,\n1###########################################...,"[['os.system(cmd)', '==', '0'], [""atoms['charg...",6,6,1.000000,0.001037,"['get_fn', 'trj', 'ref_trj', 'ref_top', 'ref_b...",37,You are a helpful bot that adds assertions to ...,7731,"[[35, 'get_fn', '!=', None, ""Function get_fn m..."
9,Dfred/concept-robot,# PyVision License\n#\n# Copyright (c) 2006-20...,\n1# PyVision License\n2#\n3# Copyright (c) 20...,"[['len(tmp)', '>', '0']]",1,1,1.000000,0.000175,"['*points', 'tmp', 'minx', 'miny', 'maxx', 'ma...",38,You are a helpful bot that adds assertions to ...,7873,"[40, '*points', '!=', 0, ""The function needs a..."
33,nanalelfe/fofe-ner,#!/home/chwang/anaconda2/envs/tensorflow/bin/p...,\n1#!/home/chwang/anaconda2/envs/tensorflow/bi...,"[['len(offsets)', '==', 'len(sent)'], ['0', '<...",4,2,0.500000,0.000328,"['logger', 'rspecifier', 'language', 'entity2c...",34,You are a helpful bot that adds assertions to ...,7908,"[[11, 'rspecifier', '!=', '', ""function LoadED..."


In [19]:
df.describe()

,asserted_lines,parsed_lines,arr,atl,num_vars,prompt_len
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,10.965000,8.525000,0.849637,0.002552,13.960000,4531.530000
std,15.496921,13.600893,0.294534,0.002973,9.514781,1732.097439
min,1.000000,1.000000,0.037736,0.000185,1.000000,1517.000000
25%,2.000000,1.000000,0.746528,0.000588,6.000000,2944.250000
50%,4.000000,3.000000,1.000000,0.001417,11.500000,4519.000000
75%,14.000000,8.000000,1.000000,0.003679,20.250000,6041.250000
max,99.000000,95.000000,2.000000,0.023152,45.000000,8140.000000


In [29]:
ind = 11
print("Showing Data for prompt_len sorted data in Part1 Ind"+str(ind))
print("\nExtracted Assertion:")
print(df.iloc[ind]["assertions"])
print("\nVariables:")
print(df.iloc[ind]["variables"])
print("\nCode:")
print(df.iloc[ind]["content"])
print("\nCode:")
print(df.iloc[ind]["unasserted"])

print("\nGPT Response:")
print(df.iloc[ind]["gpt"])

# print("\nPROMPT:")
# print(df.iloc[ind]["prompt"])

Showing Data for prompt_len sorted data in Part1 Ind11

Extracted Assertion:
[['completion', '==', 'True'], ['completion', '==', 'True']]

Variables:
['completion']

Code:
import pytest


@pytest.mark.bashcomp(cmd="munin-node-configure")
class TestMuninNodeConfigure:
    @pytest.mark.complete("munin-node-configure --libdir ")
    def test_1(self, completion):
        assert completion

    @pytest.mark.complete(
        "munin-node-configure -",
        require_cmd=True,
        xfail=(
            "! (munin-node-configure --help 2>&1 || :) "
            "| command grep -q -- '[[:space:]]-'"
        ),
    )
    def test_2(self, completion):
        assert completion


Code:

1import pytest
2
3
4@pytest.mark.bashcomp(cmd="munin-node-configure")
5class TestMuninNodeConfigure:
6    @pytest.mark.complete("munin-node-configure --libdir ")
7    def test_1(self, completion):
8
9    @pytest.mark.complete(
10        "munin-node-configure -",
11        require_cmd=True,
12        xfail=(
13    

# Step 5) Replicating for Verilog

In [126]:
import re

def extract_variables(verilog_code):
    # Regular expression to match Verilog variable declarations
    variable_pattern = r'\b(?:wire|reg|integer|real|time)\s+(.*?)\s*[,;]'

    # Find all matches
    matches = re.findall(variable_pattern, verilog_code, re.MULTILINE)

    # Remove extra whitespace and split on commas
    variables = [var.strip() for match in matches for var in match.split(',')]

    return variables

def v_extract_assertions(verilog_code):
    # Regular expression to match Verilog assertions
    assertion_pattern = r'\bassert\s*\((.*?)\)\s*;'
    
    # Find all matches
    matches = re.findall(assertion_pattern, verilog_code, re.MULTILINE)
    
    return matches

def old_v_get_assertions(code):
    ret = []
    for line in code.split('\n'):
        if 'assert' in line:
            ret.append(line.strip())
    return ret

def v_get_variables(assertions):
    ret = []
    for statement in assertions:
        ret.append(statement.split(' ')[0].strip())
    return ret

In [132]:
class VLLM_prompt:       
    def __init__(self, input_code="*Variables:\n[flag, num, i]\n*Code:\n1num = int(input(\"Enter a number: \"))  # Program to check if a number is prime or not\n2flag = False  # define a flag variable\n3\n4if num == 1:\n5    print(num, \"is not a prime number\")\n6elif num > 1: # check for factors\n7    for i in range(2, num):\n8        if (num % i) == 0:\n9            flag = True  # if factor is found, set flag to True\n10            break  # break out of loop\n11    if flag:  # check if flag is True\n12        print(num, \"is not a prime number\")\n13    else:\n14        print(num, \"is a prime number\")",
                 example_in="\n*Variables:\n[a, b]\n*Code:\n1module m (\n2    input a,\n3    b\n4);\n5  a1 :\n6endmodule\n7module m (\n8    input a,\n9    b\n10);\n11  always_comb begin\n12  end\n13endmodule\n14",
                 example_out="[5, #0, a, \"==\", b, \"checking input a and output b at ever 0th clock cycle is necessart for the code to function\"]\n\nWhich would be the same as:\nmodule m (\n    input a,\n    b\n);\n  a1 :\n  assert #0 (a == b);\nendmodule\nmodule m (\n    input a,\n    b\n);\n  always_comb begin\n    a1 : assert #0 (a == b);\n  end\nendmodule", 
                 criteria=["Assert that the function can take in all inputs necessary to complete the process",
                           "Assert that all outputs are of the proper sizes."]
                 ):
        self.criteria = criteria
        self.example = prompt_example(example_in, example_out)
        self.input_code = input_code
        
        # default params that are less likely to change
        self.intro = "You are a helpful bot that adds assertions to pieces of Verilog code."  
        self.input_format = "You will be given a string of code presented in the format:\n*Variables:\n...\n*Code:\n..."
        self.criteria_transition = "Generate assertions based on the following criteria:"
        self.output_format = "Your response should ONLY be a list of assertions in the format:\n[line_number, timing, subject_variable, condition_type, target, reasoning]"
        self.output_format_description = ["line_number is an integer referencing the line after which the assertion should be inserted",
                                          "timing is the clock cycle(s) at which the assertion is checked relative to the present cycle",
                                          "subject_variable and target can ONLY be variables present in the code, integers, booleans, or None",
                                          "condition_type can only be a value in this list: [==, >=, <=, !=]",
                                          "reasoning is a short decription of why the assertion was made"]
        self.example_transition = "Here is an example of what your input will look like and what you should return:"
        self.input_transition = "Here is the actual input you should provide assertions for:"
    
    
    
    def composite_criteria(self):
        """ return criteria as a single string"""
        ret = ""
        for i, crit in enumerate(self.criteria):
            ret += str(i+1) + ") " + crit
            if i != len(self.criteria)-1:  # ignore last instance for formatting
                ret += '\n'
        return ret
    
    def composite_output_formatting(self):
        ret = self.output_format
        for desc in self.output_format_description:
            ret += "\n -" + desc 
        return ret
    
    def prompt(self):
        """ return entire prompt"""
        return '\n'.join([self.intro, self.input_format,
                          self.criteria_transition, self.composite_criteria(), "",
                          self.composite_output_formatting(), "",
                          self.example_transition, self.example.composite(), "\n",
                          self.input_transition, self.input_code])
    
    def to_list(self):
        """ return key prompt components as a list """
        return [self.intro, self.formatting, self.criteria, self.example, self.input_code, self.prompt()]
    
    def __str__(self):
        return self.prompt()
    def __repr__(self):
        return self.prompt()
    
def v_gen_prompt(df):
    ret = []
    for i, row in tqdm(df.iterrows()):
        my_vars = row["variables"]
        my_code = row["unasserted"]
        prompt_param = "*Variables:\n" + str(row["variables"]) + "\n*Code:\n" + row["unasserted"]
        ret.append(str(VLLM_prompt(prompt_param)))
    return ret
        
        
print(str(VLLM_prompt(vdf.iloc[2]["unasserted"])))

You are a helpful bot that adds assertions to pieces of Verilog code.
You will be given a string of code presented in the format:
*Variables:
...
*Code:
...
Generate assertions based on the following criteria:
1) Assert that the function can take in all inputs necessary to complete the process
2) Assert that all outputs are of the proper sizes.

Your response should ONLY be a list of assertions in the format:
[line_number, timing, subject_variable, condition_type, target, reasoning]
 -line_number is an integer referencing the line after which the assertion should be inserted
 -timing is the clock cycle(s) at which the assertion is checked relative to the present cycle
 -subject_variable and target can ONLY be variables present in the code, integers, booleans, or None
 -condition_type can only be a value in this list: [==, >=, <=, !=]
 -reasoning is a short decription of why the assertion was made

Here is an example of what your input will look like and what you should return:
Example 

In [33]:
vdf = get_asserted_code(verilog_dir, "", False)
vdf["content_len"] = vdf["content"].apply(lambda code: len(code))
vdf = vdf.sort_values("content_len", ascending=True)
vdf["unasserted"] = vdf["content"].apply(lambda code: unassert(code, '', False))
print("vdf len", len(vdf))

# vdf["assertions"] = vdf["content"].apply(lambda code: v_extract_assertions(code))
# vdf["num_assertions"] = vdf["assertions"].apply(lambda code: len(code))
# all_prompts = len(vdf)
# vdf = vdf[vdf["num_assertions"] > 0]
# all_prompts = 100*len(vdf)/all_prompts
# print("\ndropping prompts with no exctracted assertions =>", str(all_prompts)+'%')
# print()

# vdf["variables"] = vdf["assertions"].apply(lambda code: v_get_variables(code))
# vdf["num_vars"] = vdf["variables"].apply(lambda code: len(code))

# vdf["prompt"] = v_gen_prompt(vdf)
# vdf["prompt_len"] = vdf["prompt"].apply(lambda code: len(code))
vdf

Found data at /Users/korahughes/Documents/GitHub/LLMCodeGen/Data/BigQuery/VerilogAssertions-ALL.csv
vdf len 446


,repo_name,content,content_len,unasserted
261,alainmarcel/Surelog,module dut();\n\n\nassert property ( @(posedge...,108,\nmodule dut();\n\n\n\n\nendmodule // dut\n
32,antlr/grammars-v4,"module m (\n input a,\n b\n);\n a1 :\n ...",170,"\nmodule m (\n input a,\n b\n);\n a1 :\..."
97,YosysHQ/yosys,"module top (\n\tinput clk,\n\tinput a, b\n);\n...",206,"\nmodule top (\n\tinput clk,\n\tinput a, b\n);..."
306,fabianschuiki/moore,// RUN: moore %s -e foo\n\nmodule foo;\n in...,212,\n// RUN: moore %s -e foo\n\nmodule foo;\n ...
100,YosysHQ/yosys,"module top (input clk, a, b);\n\talways @(pose...",241,"\nmodule top (input clk, a, b);\n\talways @(po..."
...,...,...,...,...
182,alainmarcel/Surelog,/**\n *\n * Name:\n * bp_cce_fsm.v\n *\n * D...,62575,\n/**\n *\n * Name:\n * bp_cce_fsm.v\n *\n *...
16,jkff90/verification,// (C) 2001-2013 Altera Corporation. All right...,86039,\n// (C) 2001-2013 Altera Corporation. All rig...
304,fabianschuiki/moore,// Author: Florian Zaruba <zarubaf@iis.ee.ethz...,171780,\n// Author: Florian Zaruba <zarubaf@iis.ee.et...
373,MarkBlanco/FPGA_Sandbox,"// (c) Copyright 2016 Xilinx, Inc. All rights...",185628,"\n// (c) Copyright 2016 Xilinx, Inc. All righ..."


In [38]:
print(vdf.iloc[0]["content"])

module dut();


assert property ( @(posedge Clock)  strap_en_i |=> ##0 !strap_en_i [*]);


endmodule // dut



In [161]:
prompt_limit = 8192
all_prompts = len(vdf)
vdf = vdf[vdf["prompt_len"] < prompt_limit]
all_prompts = 100*len(vdf)/all_prompts
print("dropping prompts over limit =>", str(all_prompts)+'%')
print(len(vdf), "left...")

dropping prompts over limit => 63.63636363636363%
70 left...


In [162]:
print("GENERATING RESPONSES")
responses = []
for prompt in tqdm(vdf["prompt"]):
    responses.append(gpt_oneshot(prompt))
vdf["gpt"] = responses
vdf.to_csv(cwd+"/verilog_prompts_withresponse3.csv") # saving data

GENERATING RESPONSES


100%|████████████████████████████████████████████████| 70/70 [26:06<00:00, 22.38s/it]
/var/folders/mv/k0krwyts3z5cfn3y2tltw0sc0000gn/T/ipykernel_38670/2562689721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vdf["gpt"] = responses


In [30]:
vdf = pd.read_csv(cwd+"/verilog_prompts_withresponse3.csv")
vdf

,Unnamed: 0,repo_name,content,content_len,unasserted,assertions,num_assertions,variables,num_vars,prompt,prompt_len,gpt
0,232,alainmarcel/Surelog,"module tb (input clock, a, b);\n\twire x, y;\n...",250,"\n1module tb (input clock, a, b);\n2\twire x, ...","['x == ($past(a, 2) ^ $past(b, 2))', 'y == (!$...",2,"['x', 'y']",2,You are a helpful bot that adds assertions to ...,1768,"[[2, '#0', 'x', '!=', None, ""Asserting that wi..."
1,1,swallat/yosys,"module top (\n input clk, rst,\n output reg ...",272,"\n1module top (\n2 input clk, rst,\n3 output...",['cnt != 15'],1,['cnt'],1,You are a helpful bot that adds assertions to ...,1868,"[7, #1, cnt, ""=="", 0, ""checking if counter is ..."
2,105,YosysHQ/yosys,module top;\n reg [0:7] mem [0:2];\n\n i...,584,\n1module top;\n2 reg [0:7] mem [0:2];\n3\n...,"[""$countbits(a, '0) == 24"", ""$countbits(a, '1)...",9,"['$countbits(a,', '$countbits(a,', '$countbits...",9,You are a helpful bot that adds assertions to ...,2006,"[6, #0, 'a', '==', 'mem[1]', ""Ensuring 'mem[1]..."
3,108,YosysHQ/yosys,// An example showing how parameters get infer...,617,\n1// An example showing how parameters get in...,"[""w0 == '0"", 'w1 == u1 ^ v1']",2,"['w0', 'w1']",2,You are a helpful bot that adds assertions to ...,2205,"[[3, #0, 'a', '==', 'b', 'It is necessary to v..."
4,39,zachjs/sv2v,"module Module(input clock, input clear, input ...",757,"\n1module Module(input clock, input clear, inp...","['1', '1', '1', '1']",4,"['1', '1', '1', '1']",4,You are a helpful bot that adds assertions to ...,2169,"[3, #0, 'y', '==', 'data', ""checking input 'da..."
...,...,...,...,...,...,...,...,...,...,...,...,...
65,254,alainmarcel/Surelog,/**\n * Name:\n * bp_be_dcache_wbuf.sv\n *...,5575,\n1/**\n2 * Name:\n3 * bp_be_dcache_wbuf.s...,"['~reset_i || num_els_r < 2\'d3) else $error(""...",1,['~reset_i'],1,You are a helpful bot that adds assertions to ...,7535,"[22, ""#0"", ""clk_i"", ""=="", 1, ""input clock (clk..."
66,172,alainmarcel/Surelog,// Copyright 2018 ETH Zurich and University of...,5665,\n1// Copyright 2018 ETH Zurich and University...,"['tck_src > 0', 'tck_dst > 0']",2,"['tck_src', 'tck_dst']",2,You are a helpful bot that adds assertions to ...,7362,"[[52, ""#0"", ""dst_mbox"", ""=="", None, ""checking ..."
67,146,alainmarcel/Surelog,// Copyright 2019 ETH Zurich and University of...,5699,\n1// Copyright 2019 ETH Zurich and University...,"[""FALL_THROUGH == 1'b0""]",1,['FALL_THROUGH'],1,You are a helpful bot that adds assertions to ...,7229,Based on the given Verilog code and assertion ...
68,162,alainmarcel/Surelog,// Copyright 2018 ETH Zurich and University of...,5806,\n1// Copyright 2018 ETH Zurich and University...,"['binary != """") else $error(""We need a preload...",1,['binary'],1,You are a helpful bot that adds assertions to ...,7069,"Based on the given code, here are some asserti..."


In [31]:
ind = 1
print("Showing Data for content_len sorted data Ind"+str(ind))
print("\nExtracted Assertion:")
print(vdf.iloc[ind]["assertions"])

print("\nVariables:")
print(vdf.iloc[ind]["variables"])

print("\nGPT Response:")
print(vdf.iloc[ind]["gpt"])

print("\nCode:")
print(vdf.iloc[ind]["content"])  # unasserted


print("\nPROMPT:")
print(vdf.iloc[ind]["prompt"])

Showing Data for content_len sorted data Ind1

Extracted Assertion:
['cnt != 15']

Variables:
['cnt']

GPT Response:
[7, #1, cnt, "==", 0, "checking if counter is reset to zero when 'rst' is true"],
[11, #1, cnt, "<=", 15, "checking if counter 'cnt' does not exceed its maximum size"]

Code:
module top (
  input clk, rst,
  output reg [3:0] cnt
);
  initial cnt = 0;

  always @(posedge clk) begin
    if (rst)
      cnt <= 0;
    else
      cnt <= cnt + 4'd 1;
  end

  always @(posedge clk) begin
    assume (cnt != 10);
    assert (cnt != 15);
  end
endmodule


PROMPT:
You are a helpful bot that adds assertions to pieces of Verilog code.
You will be given a string of code presented in the format:
*Variables:
...
*Code:
...
Generate assertions based on the following criteria:
1) Assert that the function can take in all inputs necessary to complete the process
2) Assert that all outputs are of the proper sizes.

Your response should ONLY be a list of assertions in the format:
[line_number,

## Step 3) Parse & Evaluate GPT's Response

### Step 3.1) Restore the assertion(s) generated to code and evaluate
> Metrics of evaluation, does it run? does it add to the code? is it ground-truth-like? human evaluator rank? gpt evaluator rank?

In [25]:
def get_gpt_assertions(response, code):
    """ takes in chat gpt's response and outputs its assertions as well as a string of code with said assertions in it """
    asserts = []
    parsed_code = code.split('\n')
    for line in response.split('\n'):
        line.replace('[', '').replace(']', '')
        separated = line.split(',')
        full_assert = separated[1:-1] # ommit reasoning
        
        # TODO: handle case where there are other ints in the code
        line_num = separated[0]
        num_size = len(str(line_num))
        has_found = False
        for i, line in enumerate(parsed_code):
            if line_num in line[:num_size+1]:
                parsed_code.insert(i+1, full_assert)
                asserts.append(full_assert)
                has_found = True
                break
        if not has_found:
            print("Could not find location of\n", full_assert, "\nin\n", code)
    return '\n'.join(parsed_code), asserts

example_response = tester_df.sample()
print(example_response["gpt"].iloc[0])
temp_test = get_gpt_assertions(example_response["gpt"].iloc[0], example_response["unasserted"].iloc[0])
print(temp_test)

In [ ]:
gpt_asserted_code = []  # snippets of code greated by the response assertions from gpt
gpt_assertions = []  # the decoded assertions themselves
gpt_num_assertions = []  # the number of assertions gpt generated
gpt_ratio_assertions = []   # num_gen_assertions / num_parsed_assertions
gpt_matched_assertions = []  # assertions that roughly equal ground-truth
gpt_matched_assertions_ratio = []  # num_matched_assertions / num_ground_truth_assertions

for i, row in tester_df.iterrows():
    new_code, asserts = get_gpt_assertions(row["gpt"], row["Unasserted"])
    gpt_asserted_code.append(new_code)
    gpt_assertions.append(asserts)
    gpt_num_assertions.append(len(asserts))
    gpt_ratio_assertions.append(len(asserts)/row["parsed_lines"])
    # TODO get number of matching assertions
    matched_num = ...
    gpt_matched_assertions.append(matched_num)
    gpt_matched_assertions_ratio.append(matched_num/len(asserts))
tester_df["gpt_asserted_code"] = gpt_asserted_code
tester_df["gpt_assertions"] = gpt_assertions
tester_df["gpt_num_assertions"] = gpt_num_assertions
tester_df["gpt_ratio_assertions"] = gpt_ratio_assertions
tester_df["gpt_matched_assertions"] = gpt_matched_assertions
tester_df["gpt_matched_assertions_ratio"] = gpt_matched_assertions_ratio

In [ ]:
tester = df.iloc[ind]["assertions"][1:-1].split('], [')
# print(tester)
print(df.iloc[ind]["assertions"][1:-1])
# def revive_assertion(my_list):
#     my_list[1:-1]

# for i, row in df.iterrows():
#     for list(assertion) in row["assertions"]:
#         to_find = assertion.replace('[','').replace(']','')
#         if to_find in row["gpt"]:
#             print("Found a match!")
#             print(assertion)
#             print("found at")
#             print(row["gpt"])
# print("\nDONE")

In [ ]:
""" TODO: test word-mover's distance comparison? """

## Step 4) ...